In [12]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from imgaug import augmenters as iaa
import glob

In [2]:
# 구글 드라이브 불러오기
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [57]:
file_path = '/content/drive/MyDrive/Colab Notebooks/관세청/final_dataset.xlsx'
df = pd.read_excel(file_path)

In [58]:
df.head()

,Name,Image URL,label
0,Elite 스트레이트 진 | 7 For All Mankind,https://img.mytheresa.com/652/738/100/jpeg/cat...,6204
1,로우라이즈 부츠컷 진 | Saint Laurent,https://img.mytheresa.com/652/738/100/jpeg/cat...,6204
2,Marlo 스트레이트 진 | Tove,https://img.mytheresa.com/652/738/100/jpeg/cat...,6204
3,Nedar 하이라이즈 스트레이트 진 | Loro Piana,https://img.mytheresa.com/652/738/100/jpeg/cat...,6204
4,스트레이트 진 | Toteme,https://img.mytheresa.com/652/738/100/jpeg/cat...,6204


In [59]:
df.tail()

,Name,Image URL,label
13411,NaN,NaN,6104
13412,NaN,NaN,6104
13413,NaN,NaN,6104
13414,NaN,NaN,6104
13415,NaN,NaN,6104


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13416 entries, 0 to 13415
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Name       11954 non-null  object
 1   Image URL  11954 non-null  object
 2   label      13416 non-null  int64 
dtypes: int64(1), object(2)
memory usage: 314.6+ KB


label에 따른 빈도분석

In [7]:
label_counts = df['label'].value_counts()
label_counts

label
6110    2296
6203    1941
6109    1631
6204    1584
6206    1214
6205     875
6103     734
6202     671
6201     468
6104     401
6106     401
6105     400
6101     400
6102     400
Name: count, dtype: int64

In [8]:
# 파일 경로 설정
excel_path = '/content/drive/MyDrive/Colab Notebooks/관세청/final_dataset.xlsx'  # 엑셀 파일 경로
image_folder_path = '/content/drive/MyDrive/Colab Notebooks/관세청/final_image'  # 이미지 폴더 경로
augmented_folder = '/content/drive/MyDrive/Colab Notebooks/관세청/augmented_folder'  # 증강된 이미지 저장 폴더 경로

In [9]:
# 증강된 이미지를 저장할 폴더가 없으면 생성
if not os.path.exists(augmented_folder):
    os.makedirs(augmented_folder)

**6205 : 125개 증강(13418~13542)**

In [11]:
# 엑셀 파일 읽기
df = pd.read_excel(excel_path)

# 6205 라벨에 해당하는 데이터 필터링
label_6205 = df[df['label'] == 6205]

In [15]:
# 이미지 증강 함수 정의
def augment_images(image, count, start_index):
    seq = iaa.Sequential([
        iaa.Fliplr(0.5), # 좌우 반전
        iaa.Affine(rotate=[90, 180, 270]), # 90도 단위 회전
        iaa.GaussianBlur(sigma=(0, 1.0)) # 가우시안 블러
    ])

    images = [np.array(image)] * count
    images_aug = seq(images=images)

    for i, img in enumerate(images_aug):
        img_pil = Image.fromarray(img)
        img_pil.save(os.path.join(augmented_folder, f'{start_index + i}.png'), format='PNG')

In [17]:
# 확장자를 포함한 이미지 파일 경로 패턴 생성
image_files = []
for ext in ['png', 'PNG', 'jpg', 'jpeg']:
    image_files.extend(glob.glob(os.path.join(image_folder_path, f'*.{ext}')))

# 이미지 파일명을 행 번호와 매핑 (엑셀의 행 번호가 2부터 시작하므로 1을 더해줌)
image_files_map = {int(os.path.splitext(os.path.basename(f))[0]): f for f in image_files}

# 현재 존재하는 원본 이미지 수
current_count = label_6205.shape[0]

# 증강할 총 이미지 수
target_count = 1000

# 부족한 이미지 개수
augment_count_total = target_count - current_count

# 원본 이미지당 증강할 이미지 개수 계산
augment_per_image = augment_count_total // current_count
remainder = augment_count_total % current_count

# 증강할 이미지의 초기 인덱스 설정
current_index = 13418

# 증강 수행
for idx, row in label_6205.iterrows():
    image_path = image_files_map.get(idx + 2, None)  # 엑셀의 행 번호는 2부터 시작하므로 idx에 2를 더해줌
    if image_path and os.path.exists(image_path):
        image = Image.open(image_path)

        # 원본 이미지당 증강할 이미지 수 계산
        if augment_count_total > 0:
            if remainder > 0:
                augment_count = augment_per_image + 1
                remainder -= 1
            else:
                augment_count = augment_per_image

            # 이미지 증강
            augment_images(image, augment_count, current_index)

            # 인덱스 갱신
            current_index += augment_count
            augment_count_total -= augment_count

print("이미지 증강이 완료되었습니다.")

이미지 증강이 완료되었습니다.


**6103 : 266개 증강(13543~13808)**

In [18]:
# 6103 라벨에 해당하는 데이터 필터링
label_6103 = df[df['label'] == 6103]

In [20]:
# 확장자를 포함한 이미지 파일 경로 패턴 생성
image_files = []
for ext in ['png', 'PNG', 'jpg', 'jpeg']:
    image_files.extend(glob.glob(os.path.join(image_folder_path, f'*.{ext}')))

# 이미지 파일명을 행 번호와 매핑 (엑셀의 행 번호가 2부터 시작하므로 1을 더해줌)
image_files_map = {int(os.path.splitext(os.path.basename(f))[0]): f for f in image_files}

# 현재 존재하는 원본 이미지 수
current_count = label_6103.shape[0]

# 증강할 총 이미지 수
target_count = 1000

# 부족한 이미지 개수
augment_count_total = target_count - current_count

# 원본 이미지당 증강할 이미지 개수 계산
augment_per_image = augment_count_total // current_count
remainder = augment_count_total % current_count

# 증강할 이미지의 초기 인덱스 설정
current_index = 13543

# 증강 수행
for idx, row in label_6103.iterrows():
    image_path = image_files_map.get(idx + 2, None)  # 엑셀의 행 번호는 2부터 시작하므로 idx에 2를 더해줌
    if image_path and os.path.exists(image_path):
        image = Image.open(image_path)

        # 원본 이미지당 증강할 이미지 수 계산
        if augment_count_total > 0:
            if remainder > 0:
                augment_count = augment_per_image + 1
                remainder -= 1
            else:
                augment_count = augment_per_image

            # 이미지 증강
            augment_images(image, augment_count, current_index)

            # 인덱스 갱신
            current_index += augment_count
            augment_count_total -= augment_count

print("이미지 증강이 완료되었습니다.")

이미지 증강이 완료되었습니다.


**6202 : 329개 증강(13809~14137)**

In [21]:
# 6202 라벨에 해당하는 데이터 필터링
label_6202 = df[df['label'] == 6202]

In [22]:
# 확장자를 포함한 이미지 파일 경로 패턴 생성
image_files = []
for ext in ['png', 'PNG', 'jpg', 'jpeg']:
    image_files.extend(glob.glob(os.path.join(image_folder_path, f'*.{ext}')))

# 이미지 파일명을 행 번호와 매핑 (엑셀의 행 번호가 2부터 시작하므로 1을 더해줌)
image_files_map = {int(os.path.splitext(os.path.basename(f))[0]): f for f in image_files}

# 현재 존재하는 원본 이미지 수
current_count = label_6202.shape[0]

# 증강할 총 이미지 수
target_count = 1000

# 부족한 이미지 개수
augment_count_total = target_count - current_count

# 원본 이미지당 증강할 이미지 개수 계산
augment_per_image = augment_count_total // current_count
remainder = augment_count_total % current_count

# 증강할 이미지의 초기 인덱스 설정
current_index = 13809

# 증강 수행
for idx, row in label_6202.iterrows():
    image_path = image_files_map.get(idx + 2, None)  # 엑셀의 행 번호는 2부터 시작하므로 idx에 2를 더해줌
    if image_path and os.path.exists(image_path):
        image = Image.open(image_path)

        # 원본 이미지당 증강할 이미지 수 계산
        if augment_count_total > 0:
            if remainder > 0:
                augment_count = augment_per_image + 1
                remainder -= 1
            else:
                augment_count = augment_per_image

            # 이미지 증강
            augment_images(image, augment_count, current_index)

            # 인덱스 갱신
            current_index += augment_count
            augment_count_total -= augment_count

print("이미지 증강이 완료되었습니다.")

이미지 증강이 완료되었습니다.


**6201 : 532개 증강(14138~14669)**

In [23]:
# 6201 라벨에 해당하는 데이터 필터링
label_6201 = df[df['label'] == 6201]

In [24]:
# 확장자를 포함한 이미지 파일 경로 패턴 생성
image_files = []
for ext in ['png', 'PNG', 'jpg', 'jpeg']:
    image_files.extend(glob.glob(os.path.join(image_folder_path, f'*.{ext}')))

# 이미지 파일명을 행 번호와 매핑 (엑셀의 행 번호가 2부터 시작하므로 1을 더해줌)
image_files_map = {int(os.path.splitext(os.path.basename(f))[0]): f for f in image_files}

# 현재 존재하는 원본 이미지 수
current_count = label_6201.shape[0]

# 증강할 총 이미지 수
target_count = 1000

# 부족한 이미지 개수
augment_count_total = target_count - current_count

# 원본 이미지당 증강할 이미지 개수 계산
augment_per_image = augment_count_total // current_count
remainder = augment_count_total % current_count

# 증강할 이미지의 초기 인덱스 설정
current_index = 14138

# 증강 수행
for idx, row in label_6201.iterrows():
    image_path = image_files_map.get(idx + 2, None)  # 엑셀의 행 번호는 2부터 시작하므로 idx에 2를 더해줌
    if image_path and os.path.exists(image_path):
        image = Image.open(image_path)

        # 원본 이미지당 증강할 이미지 수 계산
        if augment_count_total > 0:
            if remainder > 0:
                augment_count = augment_per_image + 1
                remainder -= 1
            else:
                augment_count = augment_per_image

            # 이미지 증강
            augment_images(image, augment_count, current_index)

            # 인덱스 갱신
            current_index += augment_count
            augment_count_total -= augment_count

print("이미지 증강이 완료되었습니다.")

이미지 증강이 완료되었습니다.


**6104 : 599개 증강(14670~15268)**

In [25]:
# 6104 라벨에 해당하는 데이터 필터링
label_6104 = df[df['label'] == 6104]

In [26]:
# 확장자를 포함한 이미지 파일 경로 패턴 생성
image_files = []
for ext in ['png', 'PNG', 'jpg', 'jpeg']:
    image_files.extend(glob.glob(os.path.join(image_folder_path, f'*.{ext}')))

# 이미지 파일명을 행 번호와 매핑 (엑셀의 행 번호가 2부터 시작하므로 1을 더해줌)
image_files_map = {int(os.path.splitext(os.path.basename(f))[0]): f for f in image_files}

# 현재 존재하는 원본 이미지 수
current_count = label_6104.shape[0]

# 증강할 총 이미지 수
target_count = 1000

# 부족한 이미지 개수
augment_count_total = target_count - current_count

# 원본 이미지당 증강할 이미지 개수 계산
augment_per_image = augment_count_total // current_count
remainder = augment_count_total % current_count

# 증강할 이미지의 초기 인덱스 설정
current_index = 14670

# 증강 수행
for idx, row in label_6104.iterrows():
    image_path = image_files_map.get(idx + 2, None)  # 엑셀의 행 번호는 2부터 시작하므로 idx에 2를 더해줌
    if image_path and os.path.exists(image_path):
        image = Image.open(image_path)

        # 원본 이미지당 증강할 이미지 수 계산
        if augment_count_total > 0:
            if remainder > 0:
                augment_count = augment_per_image + 1
                remainder -= 1
            else:
                augment_count = augment_per_image

            # 이미지 증강
            augment_images(image, augment_count, current_index)

            # 인덱스 갱신
            current_index += augment_count
            augment_count_total -= augment_count

print("이미지 증강이 완료되었습니다.")

이미지 증강이 완료되었습니다.


**6106 : 599개 증강(15269~15867)**

In [27]:
# 6106 라벨에 해당하는 데이터 필터링
label_6106 = df[df['label'] == 6106]

In [29]:
# 확장자를 포함한 이미지 파일 경로 패턴 생성
image_files = []
for ext in ['png', 'PNG', 'jpg', 'jpeg']:
    image_files.extend(glob.glob(os.path.join(image_folder_path, f'*.{ext}')))

# 이미지 파일명을 행 번호와 매핑 (엑셀의 행 번호가 2부터 시작하므로 1을 더해줌)
image_files_map = {int(os.path.splitext(os.path.basename(f))[0]): f for f in image_files}

# 현재 존재하는 원본 이미지 수
current_count = label_6106.shape[0]

# 증강할 총 이미지 수
target_count = 1000

# 부족한 이미지 개수
augment_count_total = target_count - current_count

# 원본 이미지당 증강할 이미지 개수 계산
augment_per_image = augment_count_total // current_count
remainder = augment_count_total % current_count

# 증강할 이미지의 초기 인덱스 설정
current_index = 15269

# 증강 수행
for idx, row in label_6106.iterrows():
    image_path = image_files_map.get(idx + 2, None)  # 엑셀의 행 번호는 2부터 시작하므로 idx에 2를 더해줌
    if image_path and os.path.exists(image_path):
        image = Image.open(image_path)

        # 원본 이미지당 증강할 이미지 수 계산
        if augment_count_total > 0:
            if remainder > 0:
                augment_count = augment_per_image + 1
                remainder -= 1
            else:
                augment_count = augment_per_image

            # 이미지 증강
            augment_images(image, augment_count, current_index)

            # 인덱스 갱신
            current_index += augment_count
            augment_count_total -= augment_count

print("이미지 증강이 완료되었습니다.")

이미지 증강이 완료되었습니다.


**6105 : 600개 증강(15868~16467)**

In [30]:
# 6105 라벨에 해당하는 데이터 필터링
label_6105 = df[df['label'] == 6105]

In [31]:
# 확장자를 포함한 이미지 파일 경로 패턴 생성
image_files = []
for ext in ['png', 'PNG', 'jpg', 'jpeg']:
    image_files.extend(glob.glob(os.path.join(image_folder_path, f'*.{ext}')))

# 이미지 파일명을 행 번호와 매핑 (엑셀의 행 번호가 2부터 시작하므로 1을 더해줌)
image_files_map = {int(os.path.splitext(os.path.basename(f))[0]): f for f in image_files}

# 현재 존재하는 원본 이미지 수
current_count = label_6105.shape[0]

# 증강할 총 이미지 수
target_count = 1000

# 부족한 이미지 개수
augment_count_total = target_count - current_count

# 원본 이미지당 증강할 이미지 개수 계산
augment_per_image = augment_count_total // current_count
remainder = augment_count_total % current_count

# 증강할 이미지의 초기 인덱스 설정
current_index = 15868

# 증강 수행
for idx, row in label_6105.iterrows():
    image_path = image_files_map.get(idx + 2, None)  # 엑셀의 행 번호는 2부터 시작하므로 idx에 2를 더해줌
    if image_path and os.path.exists(image_path):
        image = Image.open(image_path)

        # 원본 이미지당 증강할 이미지 수 계산
        if augment_count_total > 0:
            if remainder > 0:
                augment_count = augment_per_image + 1
                remainder -= 1
            else:
                augment_count = augment_per_image

            # 이미지 증강
            augment_images(image, augment_count, current_index)

            # 인덱스 갱신
            current_index += augment_count
            augment_count_total -= augment_count

print("이미지 증강이 완료되었습니다.")

이미지 증강이 완료되었습니다.


**6101 : 600개 증강(16468~17067)**

In [32]:
# 6101 라벨에 해당하는 데이터 필터링
label_6101 = df[df['label'] == 6101]

In [33]:
# 확장자를 포함한 이미지 파일 경로 패턴 생성
image_files = []
for ext in ['png', 'PNG', 'jpg', 'jpeg']:
    image_files.extend(glob.glob(os.path.join(image_folder_path, f'*.{ext}')))

# 이미지 파일명을 행 번호와 매핑 (엑셀의 행 번호가 2부터 시작하므로 1을 더해줌)
image_files_map = {int(os.path.splitext(os.path.basename(f))[0]): f for f in image_files}

# 현재 존재하는 원본 이미지 수
current_count = label_6101.shape[0]

# 증강할 총 이미지 수
target_count = 1000

# 부족한 이미지 개수
augment_count_total = target_count - current_count

# 원본 이미지당 증강할 이미지 개수 계산
augment_per_image = augment_count_total // current_count
remainder = augment_count_total % current_count

# 증강할 이미지의 초기 인덱스 설정
current_index = 16468

# 증강 수행
for idx, row in label_6101.iterrows():
    image_path = image_files_map.get(idx + 2, None)  # 엑셀의 행 번호는 2부터 시작하므로 idx에 2를 더해줌
    if image_path and os.path.exists(image_path):
        image = Image.open(image_path)

        # 원본 이미지당 증강할 이미지 수 계산
        if augment_count_total > 0:
            if remainder > 0:
                augment_count = augment_per_image + 1
                remainder -= 1
            else:
                augment_count = augment_per_image

            # 이미지 증강
            augment_images(image, augment_count, current_index)

            # 인덱스 갱신
            current_index += augment_count
            augment_count_total -= augment_count

print("이미지 증강이 완료되었습니다.")

이미지 증강이 완료되었습니다.


**6102 : 600개 증강(17068~17667)**

In [34]:
# 6102 라벨에 해당하는 데이터 필터링
label_6102 = df[df['label'] == 6102]

In [35]:
# 확장자를 포함한 이미지 파일 경로 패턴 생성
image_files = []
for ext in ['png', 'PNG', 'jpg', 'jpeg']:
    image_files.extend(glob.glob(os.path.join(image_folder_path, f'*.{ext}')))

# 이미지 파일명을 행 번호와 매핑 (엑셀의 행 번호가 2부터 시작하므로 1을 더해줌)
image_files_map = {int(os.path.splitext(os.path.basename(f))[0]): f for f in image_files}

# 현재 존재하는 원본 이미지 수
current_count = label_6102.shape[0]

# 증강할 총 이미지 수
target_count = 1000

# 부족한 이미지 개수
augment_count_total = target_count - current_count

# 원본 이미지당 증강할 이미지 개수 계산
augment_per_image = augment_count_total // current_count
remainder = augment_count_total % current_count

# 증강할 이미지의 초기 인덱스 설정
current_index = 17068

# 증강 수행
for idx, row in label_6102.iterrows():
    image_path = image_files_map.get(idx + 2, None)  # 엑셀의 행 번호는 2부터 시작하므로 idx에 2를 더해줌
    if image_path and os.path.exists(image_path):
        image = Image.open(image_path)

        # 원본 이미지당 증강할 이미지 수 계산
        if augment_count_total > 0:
            if remainder > 0:
                augment_count = augment_per_image + 1
                remainder -= 1
            else:
                augment_count = augment_per_image

            # 이미지 증강
            augment_images(image, augment_count, current_index)

            # 인덱스 갱신
            current_index += augment_count
            augment_count_total -= augment_count

print("이미지 증강이 완료되었습니다.")

이미지 증강이 완료되었습니다.


라벨별 증강 완료



---



엑셀에 추가된 이미지 번호만큼 label 입력하기

In [63]:
updated_file_path = '/content/drive/MyDrive/Colab Notebooks/관세청/updated_final_dataset.xlsx'

In [65]:
# 폴더 경로 설정
folder_path = os.path.dirname(updated_file_path)

# 폴더가 존재하지 않으면 생성
if not os.path.exists(folder_path):
    os.makedirs(folder_path)

In [37]:
# 증강된 이미지의 레이블 추가
augmentations = [
    (13418, 13542, 6205),
    (13543, 13808, 6103),
    (13809, 14137, 6202),
    (14138, 14669, 6201),
    (14670, 15268, 6104),
    (15269, 15867, 6106),
    (15868, 16467, 6105),
    (16468, 17067, 6101),
    (17068, 17667, 6102),
]

In [54]:
# 새로운 데이터프레임을 생성하여 추가할 행을 저장
new_rows = []

In [55]:
# 기존 데이터프레임에 새로운 행을 추가 (Name과 url은 NA로, label만 입력)
for start, end, label in augmentations:
    for i in range(start, end + 1):
        new_rows.append({'Name': 'NA', 'Image URL': 'NA', 'label': label})

In [56]:
# 새로운 데이터프레임 생성
new_df = pd.DataFrame(new_rows)

In [60]:
# 기존 데이터프레임과 새로운 데이터프레임 병합
df = pd.concat([df, new_df], ignore_index=True)

In [61]:
df.tail()

,Name,Image URL,label
17661,NA,NA,6102
17662,NA,NA,6102
17663,NA,NA,6102
17664,NA,NA,6102
17665,NA,NA,6102


In [66]:
# 엑셀 파일 업데이트
df.to_excel(updated_file_path, index=False)